<a href="https://colab.research.google.com/github/thedavidneufeld/ResearchCode/blob/main/RQI_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [51]:
import numpy as np

**Helper Functions**

In [52]:
# given an approximate eigenvalue (mu) and a matrix M
# returns (mu)*I-M
def A(mu, M):
    I = np.eye(M.shape[0])
    return mu*I-M

# calculates and returns the conditon number of M
# based on sigma_1/sigma_n
def svd_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-1]

# calculates and returns sigma_1/sigma_{n-1}
def svd_desired_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-2]

# calculates a preconditioned matrix C based on
# an approximate eigenvalue (mu), a matrix M, and vectors
# y and v
# returns C
def f(mu, M, y, v):
    # convert y and v to column vectors
    Y = y[:, np.newaxis]
    V = v[:, np.newaxis]
    # calculate (mu)*I-M
    A1 = A(mu, M)
    # calculate ((mu)*I-M)/||((mu)*I-M)||+YV^H
    C = A1/np.linalg.norm(A1, 2) + np.dot(Y, V.T.conj())
    # return C
    return C

**RQI Algorithm**

In [53]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
def RQI(M, mu, ep, it):
    # create a random vector with norm of 1
    # vector needs to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    # track condition number of (mu)I-M through iterations
    conds = np.empty(0)
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    for i in range(1, it+1):
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate the condition number of (mu)I-M
        conds = np.append(conds, svd_cond(muI_M))
        # solve the system ((mu)I-M)x=y
        x = np.linalg.solve(muI_M, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (x^T)Mx
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x, 2) <= ep*np.linalg.norm(M, 2):
            return mu, y, i, conds
    # if max iterations has been surpassed, return None values,
    # the number of iterations, and conds
    return None, None, it, conds


**RQI Algorithm with Additive Preconditioning**

In [54]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
#
# similar to regular RQI, but uses additive preprocessing
def RQIAPP(M, mu, ep, it):
    # create two random vectors with norm of 1
    # vectors need to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    v = np.random.rand(M.shape[0])
    v = v/np.linalg.norm(v)
    # track condition number of (mu)I-M, C, and the desired condition number
    conds = np.empty(0)
    conds_C = np.empty(0)
    conds_desired = np.empty(0)
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    for i in range(1, it+1):
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate C
        C = f(mu, M, y, v)
        # calculate the condition numbers
        conds = np.append(conds, svd_cond(muI_M))
        conds_C = np.append(conds_C, svd_cond(C))
        conds_desired = np.append(conds_desired, svd_desired_cond(muI_M))
        # solve the system Cx=y
        x = np.linalg.solve(C, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (y^T)My
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x, 2) <= ep*np.linalg.norm(M, 2):
            return mu, y, i, conds, conds_C, conds_desired
    # if max iterations has been surpassed, return None values,
    # the number of iterations, and conds
    return None, None, it, conds, conds_C, conds_desired

**Test and Compare Algorithms**

In [55]:
# Setup

# create a random matrix M
M = np.random.rand(25, 25)
# calculate the eigenvalues/eigenvectors of M
vals, vecs = np.linalg.eig(M)
# extract the first eigenvalue of M and subtract
# a small value to make it an approximation
mu = vals[0]-0.25

In [56]:
# Print condition number of M
svd_cond(M)

492.89911707480775

In [57]:
# Test RQI with 1e-4
result = RQI(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (12.436021796199688+0j)
Eigenvector: [0.18651386+0.j 0.17498743+0.j 0.19491603+0.j 0.19028703+0.j
 0.20561724+0.j 0.21311619+0.j 0.22658228+0.j 0.24149701+0.j
 0.20172205+0.j 0.15627504+0.j 0.22583389+0.j 0.17270275+0.j
 0.21140866+0.j 0.22391816+0.j 0.17219116+0.j 0.18521791+0.j
 0.20333882+0.j 0.2381802 +0.j 0.19962305+0.j 0.2124687 +0.j
 0.19130547+0.j 0.20717043+0.j 0.20348932+0.j 0.19687833+0.j
 0.12469496+0.j]
Iterations: 3
Condition Numbers: [5.68049704e+01 1.26064170e+03 1.47902160e+06]


In [58]:
# Test RQIAPP with 1e-4
result = RQIAPP(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers for (mu)I-M:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (12.436023210780869+0j)
Eigenvector: [0.18651385+0.j 0.17498865+0.j 0.19491676+0.j 0.19028673+0.j
 0.20561789+0.j 0.21311647+0.j 0.22658159+0.j 0.24149718+0.j
 0.20172282+0.j 0.15627425+0.j 0.2258327 +0.j 0.1727026 +0.j
 0.21140896+0.j 0.22391669+0.j 0.17219139+0.j 0.18521723+0.j
 0.20333834+0.j 0.23817996+0.j 0.19962307+0.j 0.21246892+0.j
 0.19130672+0.j 0.20717175+0.j 0.20348896+0.j 0.19687727+0.j
 0.12469587+0.j]
Iterations: 2
Condition Numbers for (mu)I-M: [  56.80497036 3123.04107751]
Condition Numbers for C: [1.9568886  1.99560102]
Desired Condition Numbers: [1.32616718 1.31887461]


In [59]:
# Test RQI with 1e-6
result = RQI(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (12.436021796191797+0j)
Eigenvector: [0.18651386+0.j 0.17498743+0.j 0.19491603+0.j 0.19028703+0.j
 0.20561724+0.j 0.21311619+0.j 0.22658228+0.j 0.24149701+0.j
 0.20172205+0.j 0.15627504+0.j 0.22583389+0.j 0.17270275+0.j
 0.21140866+0.j 0.22391816+0.j 0.17219116+0.j 0.18521791+0.j
 0.20333882+0.j 0.2381802 +0.j 0.19962305+0.j 0.2124687 +0.j
 0.19130547+0.j 0.20717043+0.j 0.20348932+0.j 0.19687833+0.j
 0.12469496+0.j]
Iterations: 3
Condition Numbers: [5.68049704e+01 2.25652906e+03 6.93870419e+06]


In [60]:
# Test RQIAPP with 1e-6
result = RQIAPP(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (12.436021796195876+0j)
Eigenvector: [0.18651386+0.j 0.17498743+0.j 0.19491603+0.j 0.19028703+0.j
 0.20561724+0.j 0.21311619+0.j 0.22658228+0.j 0.24149701+0.j
 0.20172205+0.j 0.15627504+0.j 0.22583389+0.j 0.17270275+0.j
 0.21140866+0.j 0.22391816+0.j 0.17219116+0.j 0.18521791+0.j
 0.20333882+0.j 0.2381802 +0.j 0.19962305+0.j 0.2124687 +0.j
 0.19130547+0.j 0.20717043+0.j 0.20348932+0.j 0.19687833+0.j
 0.12469496+0.j]
Iterations: 3
Condition Numbers: [5.68049704e+01 1.46512240e+03 2.04349145e+06]
Condition Numbers for C: [2.34305579 1.58682691 1.59379417]
Desired Condition Numbers: [1.32616718 1.31902667 1.31874006]


In [61]:
# Test RQI with 1e-8
result = RQI(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: None
Eigenvector: None
Iterations: 100
Condition Numbers: [5.68049704e+01 1.63229705e+03 2.82626661e+06 6.18057685e+12
 2.92127730e+16 5.98328960e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15 4.18706680e+15 8.09279747e+15 4.18706680e+15
 8.09279747e+15

In [62]:
# Test RQIAPP with 1e-8
result = RQIAPP(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (12.4360217962177+0j)
Eigenvector: [0.18651386+0.j 0.17498743+0.j 0.19491603+0.j 0.19028703+0.j
 0.20561724+0.j 0.21311619+0.j 0.22658228+0.j 0.24149701+0.j
 0.20172205+0.j 0.15627504+0.j 0.22583389+0.j 0.17270275+0.j
 0.21140866+0.j 0.22391816+0.j 0.17219116+0.j 0.18521791+0.j
 0.20333882+0.j 0.2381802 +0.j 0.19962305+0.j 0.2124687 +0.j
 0.19130547+0.j 0.20717043+0.j 0.20348932+0.j 0.19687833+0.j
 0.12469496+0.j]
Iterations: 3
Condition Numbers: [5.68049704e+01 9.24026987e+02 8.17457287e+05]
Condition Numbers for C: [1.85699441 1.82637613 1.81376941]
Desired Condition Numbers: [1.32616718 1.31919444 1.31873975]


In [63]:
# Print actual eigenvalue/eigenvector
print(vals[0], vecs[:,0])

(12.4360217961915+0j) [-0.18651386+0.j -0.17498743+0.j -0.19491603+0.j -0.19028703+0.j
 -0.20561724+0.j -0.21311619+0.j -0.22658228+0.j -0.24149701+0.j
 -0.20172205+0.j -0.15627504+0.j -0.22583389+0.j -0.17270275+0.j
 -0.21140866+0.j -0.22391816+0.j -0.17219116+0.j -0.18521791+0.j
 -0.20333882+0.j -0.2381802 +0.j -0.19962305+0.j -0.2124687 +0.j
 -0.19130547+0.j -0.20717043+0.j -0.20348932+0.j -0.19687833+0.j
 -0.12469496+0.j]
